# Deploy ESM3-open Model Package from AWS Marketplace 

---
## 1. Overview

### 1.1. <font color='orange'>Important Note:</font>

Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq">https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.

ESM3 is a frontier generative model for biology, able to jointly reason across three fundamental biological properties of proteins: sequence, structure, and function. These three data modalities are represented as tracks of discrete tokens at the input and output of ESM3. You can present the model with a combination of partial inputs across the tracks, and ESM3 will provide output predictions for all the tracks.
ESM3 is a generative masked language model. You can prompt it with partial sequence, structure, and function keywords, and iteratively sample masked positions until all positions are unmasked.

<img src="https://aws-hcls-ml.s3.amazonaws.com/public_assets_support_materials/esm3/jumpstart/img/esm3-architecture.png" alt="ESM3 Architecture" style="width:800px;"/>


The ESM3 architecture is highly scalable due to its transformer backbone and all-to-all reasoning over discrete token sequences. At its largest scale, ESM3 was trained with 1.07e24 FLOPs on 2.78 billion proteins and 771 billion unique tokens, and has 98 billion parameters.
Here we present esm3-open-small. With 1.4B parameters it is the smallest and fastest model in the family, trained specifically to be open sourced. ESM3-open is available under a non-commercial license.

This sample notebook shows you how to deploy [EvolutionaryScale - ESM3](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

> ESM3 model package support SageMaker Realtime Inference but not SageMaker Batch Transform.

### 1.2. Prerequisites
- This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
- Ensure that IAM role used has **AmazonSageMakerFullAccess** and a trust policy for `sagemaker.amazonaws.com`, as described in the <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html#sagemaker-roles-pass-role">SageMaker documentation</a>.
- To deploy this ML model successfully, ensure that you meet one of the following conditions:
    1. Your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        - **aws-marketplace:ViewSubscriptions**
        - **aws-marketplace:Unsubscribe**
        - **aws-marketplace:Subscribe**  
    2. Your AWS account has a subscription to [ESM3](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

### 1.3. Contents
1. [Overview](#1.-Overview)
2. [Subscribe to the model package](#2.-Subscribe-to-the-model-package)
3. [Create a real-time inference endpoint ](#3.-Create-a-real-time-inference-endpoint)
4. [Test endpoint](#4.-Test-endpoint)
5. [Clean up](#5.-Clean-up)


### 1.4. Usage instructions
You can run this notebook one cell at a time by pressing the Shift+Enter keys.

---
## 2. Subscribe to the model package

1. Open the model package listing page [EvolutionaryScale ESM3 Model](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

---
## 3. Create a real-time inference endpoint

To learn more about real-time inference on Amazon SageMaker, please visit the [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### 3.1. Setup

Install dependencies. For inference capabilities we will use EvolutionaryScale's `esm` package. The order of installation is important so that dependencies don't get overriden.

In [ ]:
from IPython.display import clear_output

%pip install -U esm --no-deps
%pip install -U -r requirements.txt
%pip install -U sagemaker

clear_output()

Define ESM3 model package and instance information.

In [ ]:
ESM3_PACKAGE_ID = ""
MODEL_NAME = "esm3-sm-open-v1"  # This is the open model version
INSTANCE_TYPE = "ml.g5.2xlarge"
INITIAL_INSTANCE_COUNT = 1

In [ ]:
import boto3
import sagemaker

# Create SageMaker clients
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
sagemaker_client = boto3.client("sagemaker", region_name=region)
sagemaker_runtime_client = boto3.client("sagemaker-runtime", region_name=region)

# Get SageMaker execution role
try:
    role = sagemaker.get_execution_role()
    print(f"Default SageMaker execution role: {role}")
except ValueError as e:
    print(f"Error getting default execution role: {e}")
    print(
        "You may need to specify a role explicitly or create one if not running in a SageMaker environment."
    )

# Identify model package arm
model_package_map = {
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{ESM3_PACKAGE_ID}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{ESM3_PACKAGE_ID}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{ESM3_PACKAGE_ID}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{ESM3_PACKAGE_ID}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{ESM3_PACKAGE_ID}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{ESM3_PACKAGE_ID}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{ESM3_PACKAGE_ID}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{ESM3_PACKAGE_ID}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{ESM3_PACKAGE_ID}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{ESM3_PACKAGE_ID}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{ESM3_PACKAGE_ID}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{ESM3_PACKAGE_ID}",
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{ESM3_PACKAGE_ID}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{ESM3_PACKAGE_ID}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{ESM3_PACKAGE_ID}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{ESM3_PACKAGE_ID}",
}

if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]
print(f"Model package ARN: {model_package_arn}")

### 3.2. Create a model from the subscribed model package

In [ ]:
model = sagemaker.model.ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
    enable_network_isolation=True,
    predictor_cls=sagemaker.predictor.Predictor,
)

### 3.3. Create a real-time endpoint

Note: This step will take 10-20 minutes.

In [ ]:
predictor = model.deploy(
    initial_instance_count=INITIAL_INSTANCE_COUNT,
    instance_type=INSTANCE_TYPE,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.base_serializers.JSONSerializer(),
    deserializer=sagemaker.base_deserializers.JSONDeserializer(),
)

print(f"Deployed endpoint name is {predictor.endpoint_name}")
print(f"Model name is {MODEL_NAME}")

In [ ]:
print(predictor.endpoint_name)

In [ ]:
ENDPOINT_NAME = predictor.endpoint_name

It will require several minutes to deploy the model to an endpoint

---
## 4. Test endpoint

### 4.1. Let's create a simple new protein sequence as a test for our endpoint

In [ ]:
from esm.sdk.api import ESMProtein, GenerationConfig
from esm.sdk.sagemaker import ESM3SageMakerClient
from src.esmhelpers import format_seq, quick_pdb_plot, quick_aligment_plot

model = ESM3SageMakerClient(endpoint_name=ENDPOINT_NAME, model=MODEL_NAME)

ESM3 is a generative model, so the most basic task it can accomplish is to create the sequence and structure of a new protein. All ESM3 inference requests must include sequence information, so in this case we will pass a string of "_" symbols. This is the "mask" token that indicates where we want ESM3 to fill in the blanks.

We start by generating a new protein sequence.

In [ ]:
%%time

n_masked = 64

masked_sequence = "_" * n_masked

prompt = ESMProtein(sequence=masked_sequence)
sequence_generation_config = GenerationConfig(
    track="sequence", # We want ESM3 to generate tokens for the sequence track
    num_steps=prompt.sequence.count("_") // 4, # We'll use num(mask tokens) // 4 steps to decode the sequence
    temperature=0.7, # We'll use a temperature of 0.7 to increase the randomness of the decoding process
)

# Call the ESM3 inference endpoint
generated_protein = model.generate(
    prompt,
    sequence_generation_config,
)

# View the generated sequence
print(f"Sequence length: {len(generated_protein.sequence)}")
print(format_seq(generated_protein.sequence))


## Next steps

Voila! You have a new sequence of proteins generated using ESM3 Open model. Head to the next notebook for more basic patterns.

Once  you are finished using the endpoint, you can terminate it to avoid being charged.